In [ ]:
from jira import JIRA
import matplotlib.pyplot as plt
import pandas as pd
from numpy import nan
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"




In [ ]:
jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

# Items for the EMR Cloud team

In [ ]:
jql = 'project = ECL and type not in subTaskIssueTypes() and type != epic and fixVersion = R1 and labels = Required_for_production_data and statusCategory != Done and labels != sec_arch'
issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['type'] = ''
issues['status'] = ''
issues['SP'] = 0
issues['summary'] = ''

#add issues to dataframe
for issue in issuesRaw:
    descriptionLen = 0
    if issue.fields.description : descriptionLen = len(issue.fields.description)

    assignee = None
    if issue.fields.assignee : assignee = issue.fields.assignee.displayName
        
    issues = issues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'remainingEffort': issue.fields.customfield_14500,
         'summary': issue.fields.summary,
         'descriptionLen': descriptionLen,
         'assignee': assignee
        }, ignore_index=True)

#only use remainingEffort if it's defined
def setRemainingSP(remainingEffort, SP):
    if(remainingEffort != None): return remainingEffort
    else: return SP

issues['RemainingSP'] = issues.apply(lambda x: setRemainingSP(x['remainingEffort'], x['SP']), axis=1)

issues


In [ ]:
issues.hist(column='SP', bins=20);

# Security architect issues for R1

In [ ]:
jql = 'project = ECL and type not in subTaskIssueTypes() and type != epic and fixVersion = R1 and labels = Required_for_production_data and statusCategory != Done and labels = sec_arch'
issuesRaw = jira.search_issues(jql)

secIssues = pd.DataFrame()
secIssues['key'] = ''
secIssues['type'] = ''
secIssues['status'] = ''
secIssues['SP'] = 0
secIssues['summary'] = ''

#add issues to dataframe
for issue in issuesRaw:
    descriptionLen = 0
    if issue.fields.description : descriptionLen = len(issue.fields.description)
    secIssues = secIssues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'remainingEffort': issue.fields.customfield_14500,
         'summary': issue.fields.summary,
         'descriptionLen': descriptionLen
        }, ignore_index=True)
secIssues

In [ ]:
#find all subtasks for given tasks and check if SP sum of subtasks equals SP in task
jql = 'project = ECL and type in subTaskIssueTypes() '

subtasks = pd.DataFrame()
subtasks['key'] = ''
subtasks['status'] = ''
subtasks['SP'] = 0
subtasks['parent'] = ''
subtasks['summary'] = ''

issuesRaw = jira.search_issues(jql, maxResults=False)
for issue in issuesRaw:
            subtasks = subtasks.append(
                {
                 'key': issue.key,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'parent': issue.fields.parent.key,
                 'summary': issue.fields.summary,
                }, ignore_index=True)

#subtasks

# Parent VS Subtasks

In [ ]:
#compare sum of SP in subtasks and in parent issue
subtasksAggr = subtasks.groupby(['parent']).agg({'SP':['sum'], 'parent': 'count'}, as_index=False)
subtasksAggr.columns = subtasksAggr.columns.droplevel(0)
subtasksAggr = subtasksAggr.reset_index()

print('Subtasks')
subtasksAggr


merged_issues = issues.merge(subtasksAggr, left_on='key', right_on='parent', how='left')
#show only stories larger than 5SP
merged_issues = merged_issues[(merged_issues['SP'] > 5)]
merged_issues = merged_issues.sort_values('SP', ascending=False)

print('Parent vs subtasks')
merged_issues

# Items with no descriptions

In [ ]:
noDescriptions = issues[(issues['descriptionLen'] < 10)]
noDescriptions
#noDescriptions.to_clipboard()

# Issues not moved in last 1W

In [ ]:
jql = 'project = ECL and status="In Progress" AND not status changed after -2w'

issuesRaw = jira.search_issues(jql)

notMovingIssues = pd.DataFrame()

#add issues to dataframe
for issue in issuesRaw:
    descriptionLen = 0
    if issue.fields.description : descriptionLen = len(issue.fields.description)
    notMovingIssues = notMovingIssues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'remainingEffort': issue.fields.customfield_14500,
         'summary': issue.fields.summary,
         'updated': issue.fields.updated,
         'assignee': issue.fields.assignee
            
        }, ignore_index=True)

print('Issues not moved in last 1W')
notMovingIssues

# Assigned issues

In [ ]:
jql = 'project = ECL and assignee is not empty and statusCategory != Done'
issuesRaw = jira.search_issues(jql, maxResults=False)

assignedIssues = pd.DataFrame()


#add issues to dataframe
for issue in issuesRaw:
    descriptionLen = 0
    if issue.fields.description : descriptionLen = len(issue.fields.description)

    assignee = None
    if issue.fields.assignee : assignee = issue.fields.assignee.displayName
        
    assignedIssues = assignedIssues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'remainingEffort': issue.fields.customfield_14500,
         'summary': issue.fields.summary,
         'descriptionLen': descriptionLen,
         'assignee': assignee
        }, ignore_index=True)

assignedIssues = assignedIssues.sort_values('assignee', ascending=False)
#assignedIssues
    
t = [('person', ['Nick Friday','Thomas Thornton', 'Andy Thompson', "Martin O'Neill", 'Marty Taggart', 'James Thompson', 
        'Brett Bieker', 'Pawel Mazurkiewicz', 'Anna Rogalska'])]
team = pd.DataFrame.from_items(t)

teamIssues = team.merge(assignedIssues, left_on='person', right_on='assignee', how='left')
teamIssues


# Time to finish estimations

In [ ]:
#estimated velocity from 3 planned sprints
estVelocity = (77+35+32)/3
SPSum = issues.SP.sum()
weeksToFinishEstVel = math.ceil(SPSum / estVelocity)

avgVelocity = (25+6)/2
weeksToFinishAvgVel = math.ceil(SPSum / avgVelocity)


print('Estimated velocity per sprint (iteration, week): {}'.format(estVelocity))
print('Average sprint velocity: {0}'.format(avgVelocity))
print('Sum of SP to do: {}'.format(SPSum))
print('Sprints (iterations, weeks) to finish (according to est. velocity): {}'.format(weeksToFinishEstVel))
print('Sprints (iterations, weeks) to finish (according to avg velocity): {}'.format(weeksToFinishAvgVel))
